# Label Preparation and Final Dataset Creation (Part 4)

## Objective
This notebook handles the final piece of the puzzle: the "Labels" (Target Variables). In Part 3, we built the features (inputs). Now, we need to structure the crop yield data so the model knows what to predict.

**Key Steps:**
1.  **Load Data:** Import the features from Part 3 and the raw yield data.
2.  **Clean & Pivot:** Convert the yield data from "long" format (rows) to "wide" format (columns).
3.  **Merge:** Combine the Features (X) and Labels (Y) into one final dataset.
4.  **Save:** Export the ready-to-use dataset as `XY_v2.parquet`.

In [25]:
import pandas as pd
import numpy as np

### 1. Load Features and Raw Labels
We import our feature set (`x_features_v2.parquet`) and the raw crop yield data (`label_yield.parquet`). The raw yield data is currently in a "long" format, which means different crops are stacked in rows.

In [26]:
# Load Features created in Part 3
X = pd.read_parquet('Parquet/x_features_v3.parquet')

# Load Raw Yield Data
label_yield = pd.read_parquet('Parquet/label_yield.parquet')

### 2. Standardize Crop Names
Machine learning models prefer clean, consistent names. We process the `item` column to remove special characters and spaces, converting names like "Maize (corn)" to `maize_corn`. This prevents errors during column creation.

In [27]:
label_yield['item'] = label_yield['item'].str.replace(r'[^0-9a-zA-Z ]', '', regex=True)
label_yield['item'] = label_yield['item'].str.replace(" ", "_").str.lower()

# Display sample to verify cleaning
label_yield.head()

,area,item,year,label
0,Afghanistan,maize_corn,1970-12-31,1475.7
1,Afghanistan,maize_corn,1971-12-31,1340.0
2,Afghanistan,maize_corn,1972-12-31,1565.2
3,Afghanistan,maize_corn,1973-12-31,1617.0
4,Afghanistan,maize_corn,1974-12-31,1617.0


### 3. Create Target Columns (Pivoting)
We need a separate column for each crop so we can predict them individually.
* **Current Format (Long):** One row per crop per year.
* **New Format (Wide):** One row per year, with columns like `Y_rice`, `Y_wheat`, `Y_maize`.

We use the `pivot_table` function to transform the data structure.

In [28]:
# Extract Year as integer
label_yield['year'] = pd.to_datetime(label_yield['year']).dt.year

# Pivot the table
Y = label_yield.pivot_table(
    index=['year','area'],  # Unique identifier for row
    columns='item',         # Create columns for each crop
    values='label'          # The Yield value
).reset_index()

### 4. Rename Columns
To avoid confusion between our *features* (inputs) and our *targets* (outputs), we add a prefix `Y_` to all the new crop columns. For example, the target for Rice becomes `Y_rice`.

In [29]:
# Dynamically generate column list
current_cols = Y.columns.tolist()

# Identify crop columns (those that represent items)
crop_cols = [c for c in current_cols if c not in ['year', 'area']]

# Create new column mapping
new_col_names = ['year', 'area'] + [f'Y_{c}' for c in crop_cols]
Y.columns = new_col_names

# Display structure
Y.head()

,year,area,Y_bananas,Y_barley,Y_cassava_fresh,Y_cucumbers_and_gherkins,Y_maize_corn,Y_oil_palm_fruit,Y_other_vegetables_fresh_nec,Y_potatoes,Y_rice,Y_soya_beans,Y_sugar_beet,Y_sugar_cane,Y_tomatoes,Y_watermelons,Y_wheat
0,1970,Afghanistan,NaN,1174.6,NaN,NaN,1475.7,NaN,6127.8,9536.4,1811.9,NaN,14090.9,22000.0,NaN,7229.4,956.3
1,1970,Albania,NaN,1077.8,NaN,NaN,2071.8,NaN,12278.3,5469.3,2970.5,NaN,23638.9,NaN,12333.3,NaN,1537.7
2,1970,Algeria,NaN,668.5,NaN,NaN,1023.5,NaN,4891.3,6254.2,1581.0,NaN,19719.9,NaN,9449.6,8977.0,624.5
3,1970,Angola,10000.0,NaN,3555.6,NaN,912.0,9523.8,6515.7,6296.3,1198.0,NaN,NaN,50932.6,3076.9,NaN,854.6
4,1970,Antigua_and_Barbuda,1500.0,NaN,4000.0,4615.4,2400.0,NaN,6250.0,NaN,NaN,NaN,NaN,37272.7,3437.5,NaN,NaN


### 5. Merge Features and Labels
Now we combine everything. We perform an **inner join** between our Feature table (X) and our Label table (Y) based on `year` and `area`. This ensures every row in our final dataset has both input features and a target value to learn from.

In [30]:
XY = X.merge(Y, on=['year', 'area'], how='inner')

# Output the shape to verify merge
print(f"Final dataset shape: {XY.shape}")

Final dataset shape: (6631, 75)


### 6. Save Final Dataset
We export the merged dataframe to `XY_v2.parquet`. This file contains everything needed to train the model in the next step.

In [31]:
# Save to Parquet
XY.to_parquet('Parquet/XY_v3.parquet')

In [32]:

# Prevent pandas from hiding columns
pd.set_option('display.max_columns', None)

# Show first 20 rows for Thailand
XY[XY['area'] == 'Thailand'].head(20)

,year,area,avg_yield_maize_corn_1y,avg_yield_maize_corn_2y,avg_yield_maize_corn_3y,avg_yield_other_vegetables_fresh_nec_1y,avg_yield_other_vegetables_fresh_nec_2y,avg_yield_other_vegetables_fresh_nec_3y,avg_yield_potatoes_1y,avg_yield_potatoes_2y,avg_yield_potatoes_3y,avg_yield_rice_1y,avg_yield_rice_2y,avg_yield_rice_3y,avg_yield_sugar_cane_1y,avg_yield_sugar_cane_2y,avg_yield_sugar_cane_3y,avg_yield_wheat_1y,avg_yield_wheat_2y,avg_yield_wheat_3y,avg_yield_oil_palm_fruit_1y,avg_yield_oil_palm_fruit_2y,avg_yield_oil_palm_fruit_3y,avg_yield_barley_1y,avg_yield_barley_2y,avg_yield_barley_3y,avg_yield_soya_beans_1y,avg_yield_soya_beans_2y,avg_yield_soya_beans_3y,avg_yield_sugar_beet_1y,avg_yield_sugar_beet_2y,avg_yield_sugar_beet_3y,avg_yield_watermelons_1y,avg_yield_watermelons_2y,avg_yield_watermelons_3y,avg_yield_cucumbers_and_gherkins_1y,avg_yield_cucumbers_and_gherkins_2y,avg_yield_cucumbers_and_gherkins_3y,avg_yield_tomatoes_1y,avg_yield_tomatoes_2y,avg_yield_tomatoes_3y,avg_yield_bananas_1y,avg_yield_bananas_2y,avg_yield_bananas_3y,avg_yield_cassava_fresh_1y,avg_yield_cassava_fresh_2y,avg_yield_cassava_fresh_3y,rain_annual,rain_sin,rain_cos,solar_annual,solar_sin,solar_cos,temp_annual,temp_sin,temp_cos,pesticides_lag1,fertilizer_lag1,latitude,longitude,Y_bananas,Y_barley,Y_cassava_fresh,Y_cucumbers_and_gherkins,Y_maize_corn,Y_oil_palm_fruit,Y_other_vegetables_fresh_nec,Y_potatoes,Y_rice,Y_soya_beans,Y_sugar_beet,Y_sugar_cane,Y_tomatoes,Y_watermelons,Y_wheat
5818,1982,Thailand,2353.8,2291.05,2197.466667,6233.6,6419.95,6408.166667,9987.5,8875.00,8895.866667,1952.1,1919.95,1886.933333,43488.8,36813.30,38131.800000,NaN,NaN,NaN,10116.5,9880.45,10013.966667,NaN,NaN,NaN,1052.0,1000.90,1006.466667,NaN,NaN,NaN,15123.1,15068.05,14501.033333,7516.0,8325.10,7987.566667,7373.8,8484.25,7012.533333,12307.7,12115.40,12076.933333,14273.9,14511.40,14330.400000,1115.56,-331.152235,-455.368398,NaN,NaN,NaN,25.199167,7.949383,-14.802748,NaN,18.580125,15.87,100.99,11923.1,NaN,16361.4,7021.2,2298.8,10261.8,6656.3,11898.8,1888.0,1123.6,NaN,49240.5,8655.1,15020.0,NaN
5819,1983,Thailand,2298.8,2326.30,2293.633333,6656.3,6444.95,6498.733333,11898.8,10943.15,9882.933333,1888.0,1920.05,1909.300000,49240.5,46364.65,40955.700000,NaN,NaN,NaN,10261.8,10189.15,10007.566667,NaN,NaN,NaN,1123.6,1087.80,1041.800000,NaN,NaN,NaN,15020.0,15071.55,15052.033333,7021.2,7268.60,7890.466667,8655.1,8014.45,8541.200000,11923.1,12115.40,12051.300000,16361.4,15317.65,15128.066667,1036.59,-509.811515,-330.281927,NaN,NaN,NaN,26.107500,12.035723,-17.987600,NaN,18.704576,15.87,100.99,11769.2,NaN,18655.4,7635.6,2267.4,8361.8,6695.3,14216.0,2035.1,1149.3,NaN,42288.8,11372.1,16079.6,NaN
5820,1984,Thailand,2267.4,2283.10,2306.666667,6695.3,6675.80,6528.400000,14216.0,13057.40,12034.100000,2035.1,1961.55,1958.400000,42288.8,45764.65,45006.033333,NaN,NaN,NaN,8361.8,9311.80,9580.033333,NaN,NaN,NaN,1149.3,1136.45,1108.300000,NaN,NaN,NaN,16079.6,15549.80,15407.566667,7635.6,7328.40,7390.933333,11372.1,10013.60,9133.666667,11769.2,11846.15,12000.000000,18655.4,17508.40,16430.233333,1110.93,-515.340027,-374.690205,NaN,NaN,NaN,25.770000,12.968870,-21.662971,NaN,27.797375,15.87,100.99,11884.6,NaN,14968.8,7183.2,2430.5,8694.8,6795.3,10471.0,2067.0,1275.4,NaN,44540.8,7828.6,15604.8,NaN
5821,1985,Thailand,2430.5,2348.95,2332.233333,6795.3,6745.30,6715.633333,10471.0,12343.50,12195.266667,2067.0,2051.05,1996.700000,44540.8,43414.80,45356.700000,NaN,NaN,NaN,8694.8,8528.30,9106.133333,NaN,NaN,NaN,1275.4,1212.35,1182.766667,NaN,NaN,NaN,15604.8,15842.20,15568.133333,7183.2,7409.40,7280.000000,7828.6,9600.35,9285.266667,11884.6,11826.90,11858.966667,14968.8,16812.10,16661.866667,963.27,-343.283458,-396.494198,19.637500,11.492844,-6.229845,25.647500,6.760364,-13.014972,NaN,25.613470,15.87,100.99,12153.8,NaN,13994.1,7856.4,2571.9,11445.5,7352.9,12284.1,2060.8,1284.8,NaN,47180.3,11355.1,14293.6,NaN
5822,1986,Thailand,2571.9,2501.20,2423.266667,7352.9,7074.10,6947.833333,12284.1,11377.55,12323.700000,2